# 03 - Model Training (UNet)
Train a simple UNet model for change detection.

In [ ]:
import torch, torch.nn as nn
from torch.utils.data import DataLoader, Dataset
class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc1 = nn.Sequential(nn.Conv2d(3,32,3,padding=1),nn.ReLU(),nn.MaxPool2d(2))
        self.enc2 = nn.Sequential(nn.Conv2d(32,64,3,padding=1),nn.ReLU(),nn.MaxPool2d(2))
        self.dec1 = nn.Sequential(nn.ConvTranspose2d(64,32,2,stride=2),nn.ReLU())
        self.dec2 = nn.Sequential(nn.ConvTranspose2d(32,1,2,stride=2),nn.Sigmoid())
    def forward(self,x): x=self.enc1(x); x=self.enc2(x); x=self.dec1(x); x=self.dec2(x); return x
class Dummy(Dataset):
    def __len__(self): return 10
    def __getitem__(self,idx): import torch; return torch.rand(3,128,128), torch.randint(0,2,(1,128,128)).float()
loader = DataLoader(Dummy(),batch_size=2,shuffle=True)
model=UNet();lossf=nn.BCELoss();opt=torch.optim.Adam(model.parameters(),lr=1e-3)
for e in range(3):
    for i,(x,y) in enumerate(loader):
        p=model(x);loss=lossf(p,y);opt.zero_grad();loss.backward();opt.step()
    print('Epoch',e+1,'Loss',loss.item())
torch.save(model.state_dict(),'../models/checkpoints/unet_demo.pth')